In [1]:
from generator import load_csm_1b
import torchaudio
import torch
generator = load_csm_1b()

d:\Users\kaasa\Documents\DriveBackup\GitHub\AI_Tutor\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


import error: No module named 'triton'


In [ ]:


context1= """Think of it like this imagine you're trying to describe a bunch of different objects to someone who's never seen them before you've got to use words that are unique to each object so they can understand what you mean. In computer vision we do something similar with images these are called feature vectors or descriptors because they help computers match an image to the ones it's seen many times before """
context2 = """The codebook is like a list of these feature vectors it's used to build the embeddings which are basically vector representations of each image in the codebook, Embeddings are how we map objects to a high dimensional space where objects that are similar are close together and objects that are different are far apart think of it like a big graph with images as nodes and edges between nodes, when you look at the same object multiple times they'll form a connection to each other."""
audio = generator.generate(
    text=context1,
    speaker=1,
    context=[],
    max_audio_length_ms=100_000,
)

torchaudio.save("audio.wav", audio.unsqueeze(0).cpu(), generator.sample_rate)

In [ ]:
from generator import Segment

def load_prompt_audio(audio_path: str, target_sample_rate: int) -> torch.Tensor:
    audio_tensor, sample_rate = torchaudio.load(audio_path)
    audio_tensor = audio_tensor.squeeze(0)
    # Resample is lazy so we can always call it
    audio_tensor = torchaudio.functional.resample(
        audio_tensor, orig_freq=sample_rate, new_freq=target_sample_rate
    )
    return audio_tensor

def prepare_prompt(text: str, speaker: int, audio_path: str, sample_rate: int) -> Segment:
    audio_tensor = load_prompt_audio(audio_path, sample_rate)
    return Segment(text=text, speaker=speaker, audio=audio_tensor)

In [ ]:
context1= """Think of it like this imagine you're trying to describe a bunch of different objects to someone who's never seen them before you've got to use words that are unique to each object so they can understand what you mean. In computer vision we do something similar with images these are called feature vectors or descriptors because they help computers match an image to the ones it's seen many times before """
context2 = """The codebook is like a list of these feature vectors it's used to build the embeddings which are basically vector representations of each image in the codebook, Embeddings are how we map objects to a high dimensional space where objects that are similar are close together and objects that are different are far apart think of it like a big graph with images as nodes and edges between nodes, when you look at the same object multiple times they'll form a connection to each other."""
 
SPEAKER_PROMPTS = {
    "conversational_a": {
        "text": context1,
        "audio": "audio1.wav"
    },
}

prompt_a = prepare_prompt(
        SPEAKER_PROMPTS["conversational_a"]["text"],
        1,
        SPEAKER_PROMPTS["conversational_a"]["audio"],
        generator.sample_rate
    )


generated_segments = []
prompt_segments = [prompt_a, ]

conversation = [
        {"text": context2, "speaker_id": 1},
]
partialcontext = prompt_segments + generated_segments

for utterance in conversation:
    print(f"Generating: {utterance['text']}")
    audio_tensor = generator.generate(
        text=utterance['text'],
        speaker=utterance['speaker_id'],
        context=partialcontext,
        max_audio_length_ms=30_000,
    )
    generated_segments.append(Segment(text=utterance['text'], speaker=utterance['speaker_id'], audio=audio_tensor))
    partialcontext.append(generated_segments[-1])
    partialcontext = partialcontext[1:]

# Concatenate all generations
all_audio = torch.cat([seg.audio for seg in generated_segments], dim=0)
torchaudio.save(
    "audio.wav",
    all_audio.unsqueeze(0).cpu(),
    generator.sample_rate
)
print("Successfully generated full_conversation.wav")

In [ ]:
context1= """Think of it like this imagine you're trying to describe a bunch of different objects to someone who's never seen them before you've got to use words that are unique to each object so they can understand what you mean. In computer vision we do something similar with images these are called feature vectors or descriptors because they help computers match an image to the ones it's seen many times before
"""
context2="""The codebook is like a list of these feature vectors it's used to build the embeddings which are basically vector representations of each image in the codebook, Embeddings are how we map objects to a high dimensional space where objects that are similar are close together and objects that are different are far apart think of it like a big graph with images as nodes and edges between nodes, when you look at the same object multiple times they'll form a connection to each other.""",
    
SPEAKER_PROMPTS = {
    "conversational_a": {
        "text": context1,
        "audio": "audio1.wav"
    },
    "conversational_b": {
        "text": context2,
        "audio": "audio2.wav"
    }
}

prompt_a = prepare_prompt(
        SPEAKER_PROMPTS["conversational_a"]["text"],
        1,
        SPEAKER_PROMPTS["conversational_a"]["audio"],
        generator.sample_rate
    )
prompt_b = prepare_prompt(
        SPEAKER_PROMPTS["conversational_b"]["text"],
        1,
        SPEAKER_PROMPTS["conversational_b"]["audio"],
        generator.sample_rate
    )

generated_segments = []
prompt_segments = [prompt_a, prompt_b]

conversation = [
        {"text": "Think of it like this imagine you're trying to describe a bunch of different objects to someone who's never seen them before you've got to use words that are unique to each object so they can understand what you mean, ", "speaker_id": 1},
        {"text": "In computer vision we do something similar with images these are called feature vectors or descriptors because they help computers match an image to the ones it's seen many times before, ", "speaker_id": 1},
        {"text": "The codebook is like a list of these feature vectors it's used to build the embeddings which are basically vector representations of each image in the codebook, ", "speaker_id":1},
        {"text": "Embeddings are how we map objects to a high dimensional space where objects that are similar are close together and objects that are different are far apart, ", "speaker_id": 1},
        {"text": "think of it like a big graph with images as nodes and edges between nodes when you look at the same object multiple times they'll form a connection to each other, ", "speaker_id": 1},
        {"text": "In practice embeddings are usually learned during training using some algorithm such as autoencoders or convolutional neural networks which are trained on large datasets of labeled images, ", "speaker_id": 1},
        {"text": "The idea is that by embedding all the images in a dataset we can create a kind of digital fingerprint for each object which allows computers to compare and match them with other objects even if they're not directly visible to humans.", "speaker_id":1}
    ]
partialcontext = prompt_segments + generated_segments

for utterance in conversation:
    print(f"Generating: {utterance['text']}")
    audio_tensor = generator.generate(
        text=utterance['text'],
        speaker=utterance['speaker_id'],
        context=partialcontext,
        max_audio_length_ms=25_000,
    )
    generated_segments.append(Segment(text=utterance['text'], speaker=utterance['speaker_id'], audio=audio_tensor))
    #partialcontext.append(generated_segments[-1])
    #partialcontext = partialcontext[1:]

# Concatenate all generations
all_audio = torch.cat([seg.audio for seg in generated_segments], dim=0)
torchaudio.save(
    "full_conversation.wav",
    all_audio.unsqueeze(0).cpu(),
    generator.sample_rate
)
print("Successfully generated full_conversation.wav")